In [1]:
import numpy as np 
import pandas as pd
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from pathlib import Path
import pickle as pkl
import  gc
import glob
from tqdm import tqdm 
from dateutil.relativedelta import relativedelta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import StratifiedGroupKFold
import os

In [2]:
class CFG:
    home_directory = os.path.expanduser('~/kaggle_HomeCredit/parquet_files/')
    train_data_path = os.path.join(home_directory, 'train/')
    test_data_path = os.path.join(home_directory, 'test/')
    
    train_path = os.path.join(train_data_path, 'df_merged_train.parquet')

In [3]:
df_train = pl.read_parquet(CFG.train_path)
df_train

case_id,date_decision,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,…,Diff_dateactivated_425D,Diff_dtlastpmt_581D,Diff_dtlastpmtallstes_3545839D,Diff_employedfrom_700D,Diff_firstnonzeroinstldate_307D,Diff_dateofcredend_289D,Diff_dateofcredend_353D,Diff_dateofcredstart_181D,Diff_dateofcredstart_739D,Diff_dateofrealrepmt_138D,Diff_lastupdate_1112D,Diff_lastupdate_388D,Diff_numberofoverdueinstlmaxdat_148D,Diff_numberofoverdueinstlmaxdat_641D,Diff_overdueamountmax2date_1002D,Diff_overdueamountmax2date_1142D,Diff_refreshdate_3813885D,Diff_contractdate_551D,Diff_contractmaturitydate_151D,Diff_lastupdate_260D,Diff_openingdate_857D,Diff_contractenddate_991D,Diff_openingdate_313D,Diff_birth_259D,Diff_birthdate_87D,Diff_empl_employedfrom_271D,Diff_recorddate_4527225D,Diff_deductiondate_4917603D,Diff_processingdate_168D,max_0_24_months,max_24_months,mean_0_24_months,mean_24_months,median_0_24_months,median_24_months,min_0_24_months,min_24_months
i32,datetime[ns],i32,i8,i8,f64,f64,i32,f64,i8,i8,i16,i16,i8,i16,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,i16,i8,i16,i8,i16,i8,i8,f64,i16,i8,i16,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i16,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2019-01-03 00:00:00,201901,0,0,null,null,1917,0.0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,0,0,0,0,0,0,0,null,0,0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,11874,null,475.0,null,null,null,null,null,null,null,null,null,null,null
1,2019-01-03 00:00:00,201901,0,0,null,null,3134,0.0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,0,0,0,0,0,0,0,0.0,3,0,0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,22435,null,3718.0,null,null,null,null,null,null,null,null,null,null,null
2,2019-01-04 00:00:00,201901,0,0,null,null,4937,0.0,0,0,0,0,0,0,null,null,null,null,null,null,null,null,null,null,null,0,0,0,0,0,0,0,null,0,0,0,…,null,null,null,3245.0,2071.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,16105,null,3245.0,null,null,null,null,null,null,null,null,null,null,null
3,2019-01-03 00:00:00,201901,0,0,null,null,4643,0.0,0,1,0,2,0,1,null,null,null,null,null,null,null,null,null,null,null,0,0,0,0,0,0,1,null,0,0,0,…,null,null,null,233.0,-35.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9286,null,233.0,null,null,null,null,null,null,null,null,null,null,null
4,2019-01-04 00:00:00,201901,0,1,null,null,3390,0.0,0,1,0,0,0,1,null,null,null,null,null,null,null,null,null,null,null,0,0,0,0,0,0,0,null,0,0,0,…,null,null,null,null,-35.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,9134,null,1481.0,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2703450,2020-10-05 00:00:00,202010,91,0,0.0,176561.36,3675,0.0,0,0,0,0,0,10,-23.0,-43.0,-23.0,0.0,7356.8003,null,0.0,16392.496,6750.2,"""CA""",null,0,0,0,0,0,0,0,null,0,0,0,…,355.0,27.0,27.0,null,332.0,-94.0,4703.0,5068.0,637.0,4819.0,-3.0,4759.0,4841.0,null,null,null,-12.0,nu

In [4]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    
    return df_data, cat_cols



In [5]:
class VotingModel(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)

In [6]:
df_train, cat_cols = to_pandas(df_train)

In [7]:
X = df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
y = df_train["target"]
weeks = df_train["WEEK_NUM"]

In [8]:
df_train

,case_id,date_decision,MONTH,WEEK_NUM,target,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,...,Diff_deductiondate_4917603D,Diff_processingdate_168D,max_0_24_months,max_24_months,mean_0_24_months,mean_24_months,median_0_24_months,median_24_months,min_0_24_months,min_24_months
0,0,2019-01-03,201901,0,0,NaN,NaN,1917,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019-01-03,201901,0,0,NaN,NaN,3134,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2019-01-04,201901,0,0,NaN,NaN,4937,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2019-01-03,201901,0,0,NaN,NaN,4643,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2019-01-04,201901,0,1,NaN,NaN,3390,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526654,2703450,2020-10-05,202010,91,0,0.0,176561.36,3675,0.0,0,...,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1526655,2703451,2020-10-05,202010,91,0,0.0,301276.47,7088,6191.6,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1526656,2703452,2020-10-05,202010,91,0,0.0,14232.40,7788,0.0,0,...,153.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1526657,2703453,2020-10-05,202010,91,0,0.0,197371.58,1195,2827.2,0,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X = df_train.drop(columns=["target", "case_id", "WEEK_NUM"])
y = df_train["target"]
weeks = df_train["WEEK_NUM"]

cv = StratifiedGroupKFold(n_splits=5, shuffle=False)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 8,
    "learning_rate": 0.05,
    "n_estimators": 1000,
    "colsample_bytree": 0.8, 
    "colsample_bynode": 0.8,
    "verbose": -1,
    "random_state": 42,
    #"device": "gpu",
}

fitted_models = []

for idx_train, idx_valid in cv.split(X, y, groups=weeks):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[lgb.log_evaluation(100), lgb.early_stopping(100)]
    )

    fitted_models.append(model)

model = VotingModel(fitted_models)

DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes.Float64DType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int32DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int16DType'>, <class 'numpy.dtypes.Int16DType'>, <class 'numpy.dtypes.Int8DType'>, <class 'numpy.dtypes.Int16DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'numpy.dtypes.Float64DType'>, <class 'n